In [1]:
import os
import re
import pandas as pd
import numpy as np
from dotenv import load_dotenv
from sentence_transformers import SentenceTransformer, CrossEncoder
from pymilvus import connections, Collection, utility
from huggingface_hub import login

### **Connect to Milvus Database**

In [2]:
load_dotenv()
login(os.getenv("HF_TOKEN"))

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [3]:
connections.connect(host="localhost", port="19530")
print("Connected to Milvus server")

Connected to Milvus server


In [4]:
# Check collection
COLLECTION_NAME = "coffee_embeddings"
if not utility.has_collection(COLLECTION_NAME):
    print(f"Collection {COLLECTION_NAME} does not exist")
    exit(1)

In [5]:
collection = Collection(COLLECTION_NAME)
collection.load()
print(f"Loaded collection {COLLECTION_NAME}")

Loaded collection coffee_embeddings


### **Preprocess text and Create embedding vector**

In [6]:
# Preprocess text function
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"[^\w\s]", "", text)  # Remove punctuation
    text = re.sub(r"\d+", "", text)      # Remove numbers
    text = re.sub(r"\s+", " ", text)     # Remove extra spaces
    return text.strip()

# Create embedding function 
def emb_text(text):
    embed_model = SentenceTransformer("keepitreal/vietnamese-sbert")
    text = preprocess_text(text)
    embeddings = embed_model.encode(text, convert_to_numpy=True, show_progress_bar=False)
    return embeddings

In [ ]:
def print_results(results, ascending=True):
    if not results or len(results[0]) == 0:
        print("There are no results")
        return
    
    sorted_results = sorted(results[0], key=lambda x: x.distance, reverse=not ascending)
    
    print("Top matching results:")
    for result in sorted_results:
        print(f"ProductID: {result.entity.get('ProductID')}")
        print(f"Title: {result.entity.get('Title')}")
        print(f"Price: {result.entity.get('Price')}")
        print(f"Description: {result.entity.get('Description')}")
        print(f"Main category: {result.entity.get('Main_category')}")
        print(f"Sub category: {result.entity.get('Sub_category')}")
        print(f"Image URL: {result.entity.get('Image_url')}")
        print(f"Distance: {result.distance:.4f}")
        print("-" * 50)

#### **1. Vector Search**

In [ ]:
# Example query
query = "Latte Classic"
query_embedding = emb_text(query)

# Vector search (can be called ANN search)
search_params = {"metric_type": "L2", "params": {"nprobe": 6}}
results = collection.search(
    data=[query_embedding],
    anns_field="embedding",
    param=search_params,
    limit=5,
    expr='Main_category == "Cà phê" and Sub_category == "Cà phê máy"',
    output_fields=["Title", "Price", "Description", "Main_category", "Sub_category", "Image_url"]
)

print_results(results, ascending=False)

Top matching results:
ProductID: 6
Title: Latte Hazelnut
Price: 59000
Description: Latte Hazelnut là sự kết hợp giữa Espresso đậm đà, sữa tươi béo ngậy, và syrup hạt phỉ ngọt nhẹ, mang đến một hương vị thơm lừng, béo ngậy và mát lạnh. Thức uống này mang đến sự hòa quyện hoàn hảo giữa cà phê và hương vị hạt phỉ, tạo nên một trải nghiệm dễ chịu và thơm ngon.
Main category: Cà phê
Sub category: Cà phê máy
Image URL: http://product.hstatic.net/1000075078/product/1746441372_halzenut-latte_3ef994056c3b4765a600e853f6cca72d_grande.png
Distance: 118.4600
--------------------------------------------------
ProductID: 4
Title: Latte Bạc Xỉu
Price: 49000
Description: Latte Bạc Xỉu là sự kết hợp giữa Espresso, sữa tươi béo ngậy và cà phê phin đậm đà, tạo ra một hương vị vừa ngọt nhẹ, vừa đậm đà, dễ uống và mát lạnh.
Main category: Cà phê
Sub category: Cà phê máy
Image URL: http://product.hstatic.net/1000075078/product/1746441267_latte-bac-xiu_a6fe420fef6f4ec7a2e4f87e9e03ef4b_grande.png
Distance: 117

#### **2. Optimization**

**Case 1: If the customer directly asks to buy that item, then respond accurately.**

In [7]:
# Category mappings
mappings = {
    'Cà phê': {
        'Cà phê máy': ['Latte Classic', 'Latte Bạc Xỉu', 'Latte Coconut', 'Latte Hazelnut', 'Latte Caramel', 'Latte Almond', 'Latte Nóng'],
        'Cà phê phin': ['Bạc Xỉu Foam Dừa', 'Bạc Xỉu Caramel Muối', 'Đường Đen Sữa Đá','Bạc Xỉu Nóng',
                        'Bạc Xỉu', 'Cà Phê Sữa Đá', 'Cà Phê Đen Đá', 'Cà Phê Sữa Nóng', 'Cà Phê Đen Nóng'],
        'Cold Brew': ['Cold Brew Kim Quất', 'Cold Brew Sữa Tươi', 'Cold Brew Truyền Thống'],
        'A-Mê': ['A-Mê Tuyết Quất', 'A-Mê Tuyết Mơ', 'A-Mê Tuyết Đào', 'A-Mê Quất', 'A-Mê Mơ', 'A-Mê Đào', 'A-Mê Classic', 'Americano Nóng'],
        'Espresso': ['Espresso Đá','Cappuccino Đá', 'Caramel Macchiato Đá', 'Cappuccino Nóng', 'Caramel Macchiato Nóng', 'Espresso Nóng']
    },

    'Thức uống đá xay': {
        'Đá xay': ['Frosty Cà Phê Đường Đen', 'Frosty Caramel Arabica', 'Frosty Bánh Kem Dâu', 'Frosty Phin-Gato', 'Frosty Trà Xanh',
                   'Frosty Cà Phê Đường Đen', 'Frosty Caramel Arabica', 'Frosty Phin-Gato', 'Frosty Trà Xanh', 'Frosty Bánh Kem Dâu'],
        'Đá xay có lớp whipping cream': ['Frappe Choco Chip', 'Frappe Hazelnut', 'Frappe Caramel', 'Frappe Almond', 'Frappe Espresso', 'Frappe Coconut Coffee', 'Frappe Matcha'],
    },

    'Matcha': {
        '': ['Matcha Yuzu Đá Xay', 'Matcha Yuzu', 'Matcha Đào Đá Xay', 'Matcha Đào', 'Matcha Okinawa Trân Châu Hoàng Kim', 'Matcha Sữa Dừa Đá Xay', 'Matcha Sữa Dừa',
            'Matcha Latte', 'Matcha Tinh Khiết', 'Trà Xanh - Xinh Chẳng Phai', 'Trà Xanh - Yêu Chẳng Phai', 'Matcha Latte Tây Bắc Sữa Yến Mạch', 'Trà Xanh Tây Bắc',
            'Matcha Latte Tây Bắc Sữa Yến Mạch (Nóng)', 'Trà Xanh Nước Dừa', 'Trà Xanh Nước Dừa Yuzu', 'Matcha Latte Tây Bắc (Nóng)', 'Matcha Latte Tây Bắc']
    },

    'Trà trái cây - Hi Tea': {
        'Hi Tea': ['Hi-Tea Yuzu Kombucha', 'Hi-Tea Đào Kombucha', 'Hi-Tea - Xinh Chẳng Phai', 'Hi-Tea Đào', 'Hi-Tea Vải', 'Hi-Tea Yuzu Trân Châu', 'Hi-Tea - Yêu Chẳng Phai',
                   'Hi-Tea Đá Tuyết Mận Muối Trân Châu', 'Hi-Tea Dâu Tây Mận Muối Trân Châu', 'Hi-Tea Kim Quất Bưởi Hồng Mandarin', 'Dâu Phô Mai'],
        'Trà trái cây': ['Oolong Tứ Quý Sen',  'Oolong Tứ Quý Sen (Nóng)', 'Oolong Tứ Quý Dâu Trân Châu', 'Oolong Tứ Quý Vải',
                         'Oolong Tứ Quý Kim Quất Trân Châu', 'Trà Đào Cam Sả - Nóng', 'Trà Đào Cam Sả - Đá', 'Oolong Berry']
    },

    'Trà sữa': {
        '': ['Trà sữa Oolong Nướng Trân Châu', 'Trà Đen Macchiato', 'Hồng Trà Sữa Trân Châu', 'Hồng Trà Sữa Nóng',
            'Trà Sữa Oolong Tứ Quý Sương Sáo', 'Trà Sữa Oolong Nướng Sương Sáo','Trà Đào - Yêu Chẳng Phai',
            'Trà Sữa Oolong BLao', 'Trà sữa Oolong Nướng (Nóng)', 'Chocolate Nóng', 'Chocolate Đá']
    },

    'Bánh': {
        'Bánh ngọt': ['Mochi Kem Trà Sữa Trân Châu', 'Mochi Kem Matcha', 'Mochi Kem Chocolate', 'Mochi Kem Việt Quất', 'Mochi Kem Phúc Bồn Tử',
                      'Mousse Matcha', 'Mousse Tiramisu', 'Mousse Gấu Chocolate', 'Matcha Burnt Cheesecake', 'Burnt Cheesecake', 'Mít Sấy', 'Butter Croissant Sữa Đặc'],
        'Bánh mặn': ['Bánh Mì Que Bò Nấm Xốt Bơ', 'Bánh Mì Que Chà Bông Phô Mai Bơ Cay', 'Bánh Mì Que Pate Cột Đèn', 'Chà Bông Phô Mai', 'Croissant trứng muối', 'Butter Croissant']
    },

    'Đồ ăn chế biến': {
        '': ['Spaghetti Bò Bằm', 'Cơm Chiên Hải Sản']
    },

    'Cà phê gói mang đi': {
        '': ['Cà Phê Đen Đá Túi (30 gói x 16g)', 'Cà Phê Đen Đá Hộp (14 gói x 16g)', 'Cà Phê Hoà Tan Đậm Vị Việt (18 gói x 16 gam)',
            'Cà Phê Sữa Đá Hòa Tan Túi 25x22G', 'Cà Phê Rang Xay Original 1 250G', 'Cà Phê Sữa Đá Hòa Tan (10 gói x 22g)', 'Cà Phê Nguyên Hạt Arabica TCH (200gr)']
    }
}

In [8]:
# Function to determine if the query is an exact item
def is_exact_item(query):
    query = query.strip().lower()
    for main_cat, sub_cats in mappings.items():
        for sub_cat, items in sub_cats.items():
            for item in items:
                if item.lower() in query:
                    return True, item           # Return True and the original item name
    return False, None

In [9]:
# Function to search exact item
def search_exact_item(query):
    is_exact, exact_title = is_exact_item(query)
    if not is_exact:
        return None, None

    expr = f'Title == "{exact_title}"'
    
    try: 
        search_results = collection.search(
            data = [emb_text(preprocess_text(exact_title))],
            anns_field="embedding",
            param={"metric_type": "L2", "params": {"nprobe": 8}},
            limit=1,
            expr=expr,
            output_fields=["Title", "Price", "Description", "Image_url"]
        )
    except Exception as e:
        print(f"Error search: {e}")
        return None, None
    
    # print(f"Search results: {search_results}")
    
    retrieval_results = [
        {
            "title": hit.entity.get("Title"),
            "description": hit.entity.get("Description"),
            "price": hit.entity.get("Price"),
            "image_url": hit.entity.get("Image_url"),
        }
        for hit in search_results[0]
    ]
    return retrieval_results

In [50]:
query1 = "Cho tôi 1 ly Matcha Latte đi"
retrieval1 = search_exact_item(query1)

In [53]:
retrieval1

[{'title': 'Matcha Latte',
  'description': 'Matcha Nhật Bản hảo hạng kết hợp sữa tươi mịn màng, cân bằng vị umami thanh nhẹ và độ béo dịu, mang đến thức uống thơm ngon, đầy năng lượng.',
  'price': '55000',
  'image_url': 'http://product.hstatic.net/1000075078/product/1745246722_matcha-latte_805657f9e18948e6a6218bbce3e8fbe4_grande.png'}]

#### **Case 2: When the user asks a general question about a type of beverage/food**

In [10]:
sub_category_keywords = {
    'Cà phê máy': ['pha máy', 'latte'],
    'Cà phê phin': ['phin', 'bạc xỉu', 'đen đá', 'sữa đá'],
    'A-Mê': ['a-mê', 'americano'],
    'Espresso': ['espresso', 'cappuccino', 'caramel'],
    'Đá xay': ['đá xay', 'frosty'],
    'Đá xay có lớp whipping cream': ['frappe', 'whipping cream'],
    'Matcha': ['matcha', 'trà xanh'],
    'Trà trái cây': ['trà trái cây', 'oolong', 'đào cam sả'],
    'Trà sữa': ['trà sữa', 'hồng trà', 'chocolate'],
    'Bánh ngọt': ['bánh ngọt', 'mochi kem', 'mousse', 'cheesecake', 'croissant sữa đặc'],
    'Bánh mặn': ['bánh mặn', 'bánh mì', 'croissant trứng muối', 'chà bông']
}

# Function to determine category from query
def get_category_from_query(query):
    query = query.lower()
    
    # 1. Kiểm tra từ khóa gợi ý sub_category
    for main_cat, sub_cats in mappings.items():
        for sub_cat in sub_cats.keys():
            if sub_cat:
                # Kiểm tra từ khóa gợi ý
                if sub_cat in sub_category_keywords:
                    for keyword in sub_category_keywords[sub_cat]:
                        if keyword in query:
                            return main_cat, sub_cat
    
    # 2. Kiểm tra trực tiếp sub_category
    for main_cat, sub_cats in mappings.items():
        for sub_cat in sub_cats.keys():
            if sub_cat and sub_cat.lower() in query:
                return main_cat, sub_cat
    
    # 3. Kiểm tra trực tiếp main_category
    for main_cat in mappings.keys():
        if main_cat.lower() in query:
            return main_cat, None
    
    # 4. Kiểm tra keyword theo từng item (fallback)
    for main_cat, sub_cats in mappings.items():
        for sub_cat, items in sub_cats.items():
            for item in items:
                if item.lower() in query:
                    return main_cat, sub_cat    
    return None, None

In [27]:
def general_search_and_recommend(query, limit):
    preprocessed_query = preprocess_text(query)
    query_embedding = emb_text(preprocessed_query)
    
    main_cat, sub_cat = get_category_from_query(preprocessed_query)
    # print(main_cat, sub_cat)
    
    expr = f''
    if main_cat and sub_cat:
        expr += f'Main_category == "{main_cat}" and Sub_category == "{sub_cat}"'
    elif main_cat:
        # Nếu chưa có sub_category trong query
        if sub_cat is None:
            sub_list = list(mappings[main_cat].keys())
            
            # Nếu có nhiều sub_category => trả về list sub
            if len(sub_list) > 1: 
                if len(sub_list) == 2:
                    final_ans = f"{sub_list[0]} và {sub_list[1]}"
                else:
                    final_ans = ", ".join(sub_list[:-1]) + f" và {sub_list[-1]}"
                return f"Quán mình có các loại {main_cat.lower()} như: {final_ans}"
            
            # Nếu chỉ có 1 sub_category (hoặc rỗng) => recommend món trực tiếp
            else:
                expr = f'Main_category == "{main_cat}"'

    try:
        search_results = collection.search(
            data=[query_embedding],
            anns_field="embedding",
            param={"metric_type": "L2", "params": {"nprobe": 8}},
            limit=limit,
            expr=expr,
            output_fields=["Title", "Price", "Description", "Image_url"]
        )
    except Exception as e:
        print(f"Search error: {e}")
        return None, None
    
    # Parse retrieval results
    retrieval_results = [
        {
            "title": hit.entity.get("Title"),
            "description": hit.entity.get("Description"),
            "price": hit.entity.get("Price"),
            "image_url": hit.entity.get("Image_url"),
        }
        for hit in search_results[0]
    ]
    
    # 3. Rarank with Cross-Encoder
    rerank_model = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")
    if retrieval_results:
        pairs = [(query, f"{r['title']} {r['description']}") for r in retrieval_results]
        scores = rerank_model.predict(pairs)
        reranked_results = sorted(zip(retrieval_results, scores), key=lambda x: x[1], reverse=True)
    else: 
        return None
    
    # 4. Return final retrieval
    final_retrieval = [r for r, s in reranked_results if r in retrieval_results][:limit]
    return final_retrieval

In [16]:
query3 = "Quán mình có loại bánh mì nào ngon nhỉ?"
retrieval3 = general_search_and_recommend(query3, limit=5)

Bánh Bánh mặn


In [17]:
retrieval3

[{'title': 'Bánh Mì Que Bò Nấm Xốt Bơ',
  'description': 'Bò mềm thấm vị, nấm đông cô dai giòn. Rộp rộp vỏ bánh giòn rụm nóng hổi, thêm ngấu nghiến với xốt bơ béo bùi ngon số dzách.',
  'price': '22000',
  'image_url': 'http://product.hstatic.net/1000075078/product/1737355257_bmq-bo-nam_160a1e2a36b14f89b3ef0c61cda229ad_grande.png'},
 {'title': 'Butter Croissant',
  'description': 'Cắn một miếng, vỏ bánh ngàn lớp giòn thơm bơ béo, rồi mịn tan trong miệng. Cực dính khi nhâm nhi Butter Croissant với cà phê hoặc chấm cùng các món nước có foam trứng của Nhà',
  'price': '29000',
  'image_url': 'http://product.hstatic.net/1000075078/product/1737355250_croissant_edd90f1e46f84bcb978bdfc6a0fa0141_grande.png'},
 {'title': 'Bánh Mì Que Pate Cột Đèn',
  'description': 'Aiiiii Pate Cột Đèn đậm đà thơm béo hônggg? Rộp rộp vỏ bánh nóng hổi giòn rụm, thêm ngấu nghiến với pate cùng xốt bơ trứng đẫm vị.',
  'price': '19000',
  'image_url': 'http://product.hstatic.net/1000075078/product/1737355276_bmq-pa

#### **3. Determine to choose cases**

In [28]:
def handle_user_query(query, limit=5):
    # Case 1: Check exact item
    is_exact, exact_item = is_exact_item(query)
    if is_exact:
        return search_exact_item(query)   # Trả đúng món
    
    # Case 2: General recommend
    return general_search_and_recommend(query, limit)

In [19]:
query5 = "Quán mình có những loại bánh ngọt nào nhỉ?"
retrieval5 = handle_user_query(query5)

Bánh Bánh ngọt


### **Prompt Augmentation**

In [29]:
def get_relevant_chunk(query):
    results = handle_user_query(query, limit=5)
    
    # Case 2: Nếu trả về chuỗi danh sách sub_category
    if isinstance(results, str):
        return results
    
    if results:
        if len(results) == 1:
            # Case 1: Exact match
            item = results[0]
            context = (
                f"Tên: {item['title']}\n"
                f"Giá: {item['price']} VNĐ\n"
                f"Mô tả: {item['description']}"
            )
            return context
        else:
            context = ""
            for item in results:
                context += (
                    f"Tên: {item['title']}\n"
                    f"Giá: {item['price']} VNĐ\n"
                    f"Mô tả: {item['description']}\n"
                    f"----\n"
                )
            return context.strip()
    
    return "Quán của mình hiện không bán món nước này. Anh/chị muốn thử món nào khác không?"

In [30]:
def make_prompt(query, context):
    # Xác định case để tạo instruction phù hợp
    is_exact, _ = is_exact_item(query)
    case = 1 if is_exact else 2
    
    if case == 1:
        instruction = (
            "Hãy trả lời một cách ấm áp, nhẹ nhàng như một cuộc hội thoại, tập trung vào món mà khách hàng hỏi. "
            "Giới thiệu món, mô tả lợi ích dựa trên context, và hỏi thêm nếu cần."
        )
    else:
        instruction = (
            "Hãy trả lời một cách thân thiện, tự nhiên. "
            "Nếu context là danh sách loại món, liệt kê chúng và hỏi khách muốn loại nào cụ thể. "
            "Nếu context là danh sách món, giới thiệu ngắn gọn từng món và khuyến khích khách chọn."
        )
        
    return (
        f"Query: {query}\n\n"
        f"Context: \n{context}\n\n"
        f"Answer: {instruction}"
    )

In [31]:
import google.generativeai as genai

chat_history = []

system_prompt_content = (
    "Bạn là một chatbot của cửa hàng bán cà phê và các loại đồ uống khác. ",
    "Vai trò của bạn là hỗ trợ khách hàng trong việc tìm hiểu về các sản phẩm và dịch vụ của cửa hàng, ",
    "cũng như tạo một trải nghiệm mua sắm dễ chịu và thân thiện. ",
    "Bạn cũng có thể trò chuyện với khách hàng về các chủ đề không liên quan đến sản phẩm như thời tiết, sở thích cá nhân, ",
    "và những câu chuyện thú vị khác để tạo sự gắn kết. ",
    "Hãy luôn giữ thái độ lịch sự và chuyên nghiệp. ",
    "Nếu khách hàng hỏi về sản phẩm cụ thể, hãy cung cấp thông tin chi tiết và gợi ý các lựa chọn phù hợp. ",
    "Nếu khách hàng trò chuyện về các chủ đề không liên quan đến sản phẩm, hãy tham gia vào cuộc trò chuyện một cách vui vẻ và thân thiện.",
    "Một số điểm bạn cần lưu ý:\n",
    "1. Đáp ứng nhanh chóng và chính xác, sử dụng xưng hô là 'Mình và bạn'\n",
    "2. Giữ cho cuộc trò chuyện vui vẻ và thân thiện.\n",
    "3. Cung cấp thông tin hữu ích về quán và dịch vụ của cửa hàng.\n",
    "4. Giữ cho cuộc trò chuyện mang tính chất hỗ trợ và giúp đỡ.\n",
    "Hãy làm cho khách hàng cảm thấy được chào đón và quan tâm!"
)

### **Generate Answer**

In [ ]:
def generate_answer(query, chat_history, limit=5):
    genai.configure(api_key=os.getenv("GEMINI_API_KEY"))
    model = genai.GenerativeModel('gemini-1.5-flash')
    
    # Take context from get_relevant_chunk
    context = get_relevant_chunk(query)
    
    # Create prompt
    prompt = make_prompt(query, context)
    
    # Apppend the prompt to chat history
    if len(chat_history) > 10:
        chat_history = chat_history[-10:]
    chat_history.append(f'User: {prompt}')
    
    # Combine system message to chat history
    full_prompt = f"{system_prompt_content}\n\n" + "\n".join(chat_history) + f"\n\n{prompt}\nAssistant:"
    
    # Generate response
    try:
        response = model.generate_content(full_prompt)
        chat_history.append(f"Assistant: {response.text}")
        return response.text
    except Exception as e:
        print(f"Error generating response: {e}")
        return "Mình xin lỗi vì gặp chút trục trặc. Bạn có thể hỏi lại hoặc thử món khác nhé!"

In [33]:
def main():
    # query = "Quán mình có cà phê đen đá không? Cho tôi một ly cà phê đen đá không đường"
    # query = "Quán mình có bán những loại bánh ngọt nào vậy?"
    query = "Quán mình có bán những loại cà phê nào vậy? Có thể cho tôi biết best seller của quán không?"
    
    print(f"\nQuery: {query}")
    answer = generate_answer(query, chat_history, limit=5)
    print("Answer:", answer)
    print("Chat history:", chat_history[-2:] if len(chat_history) >= 2 else chat_history) 

if __name__ == "__main__":
    main()


Query: Quán mình có bán những loại cà phê nào vậy? Có thể cho tôi biết best seller của quán không?
Answer: Chào bạn! Quán mình có rất nhiều loại cà phê ngon đó nha!  Mình có cà phê máy thơm ngon, đậm đà, cà phê phin truyền thống, Cold Brew thơm mát, A-Mê độc đáo và Espresso mạnh mẽ.

Best seller của quán hiện nay là Cold Brew đấy bạn ạ!  Nó có vị cà phê êm dịu, không đắng gắt, rất dễ uống, đặc biệt thích hợp cho những ngày hè nóng bức.  

Bạn thấy loại nào hợp khẩu vị nhất nè?  Mình có thể tư vấn thêm cho bạn nếu bạn muốn đó!

Chat history: ['User: Query: Quán mình có bán những loại cà phê nào vậy? Có thể cho tôi biết best seller của quán không?\n\nContext: \nQuán mình có các loại cà phê như: Cà phê máy, Cà phê phin, Cold Brew, A-Mê và Espresso\n\nAnswer: Hãy trả lời một cách thân thiện, tự nhiên. Nếu context là danh sách loại món, liệt kê chúng và hỏi khách muốn loại nào cụ thể. Nếu context là danh sách món, giới thiệu ngắn gọn từng món và khuyến khích khách chọn.', 'Assistant: Chào 